In [1]:
import os
import mne
import numpy as np
from fooof import FOOOFGroup
from fooof.data import FOOOFSettings

C:\Users\Christian\AppData\Local\Temp\ipykernel_16980\725504202.py:5: DeprecationWarning: 
The `fooof` package is being deprecated and replaced by the `specparam` (spectral parameterization) package.
This version of `fooof` (1.1) is fully functional, but will not be further updated.
New projects are recommended to update to using `specparam` (see Changelog for details).
  from fooof import FOOOFGroup


In [2]:
## Subjects and Sessions to include
EEG_INFO = {
    'SUBJ_NUMS': ['10002', '10003', '10004', '10008', '10011', '10012', '10016', '10019', 
                  '10021', '10022', '10023', '10024', '10028', '10031', '10032', '10034', 
                  '10036', '10037', '10039', '10040', '10046', '10048', '10056', '10058'],
    
    'SESS_NUMS' : ['W00', 'W00_2', 'W00_3', 'W00_4',
                    'W08', 'W08_2', 'W08_3', 'W08_4',
                    'W12', 'W12_2', 'W12_3', 'W12_4',
                    'W20', 'W20_2', 'W20_3', 'W20_4'],
}
SUBJ_NUMS = EEG_INFO['SUBJ_NUMS']
SESS_NUMS = EEG_INFO['SESS_NUMS']

In [4]:
## Directory paths
epoch_load_path = '02_Epoch_EEG/Concatenated'
fooof_save_path = '03_FOOOF_EEG/Concatenated'

In [3]:
###############################################################################
# Specparam settings (FOOOF - Fitting Oscillations & One Over F)
###############################################################################

# Define the frequency range for fitting
FIT_RANGE = (.5, 13)  # Fit the model from 0.5 Hz to 13 Hz

# Configure FOOOF settings
FOOOF_SETTINGS = FOOOFSettings(
    peak_width_limits=[1, 12],  # Minimum and maximum width of peaks (in Hz)
    max_n_peaks=6,              # Maximum number of peaks to fit
    min_peak_height=0,          # Minimum height of peaks to consider
    peak_threshold=2,           # Threshold for detecting peaks, in standard deviations
    aperiodic_mode='fixed',     # Use fixed aperiodic component
)


In [7]:
## Loop through each session file
for sub_ind, sub in enumerate(SUBJ_NUMS):  # For each subject
    for sess_ind, sess in enumerate(SESS_NUMS):  # For each session
        
        ###############################################################################
        # Compute PSD (Power Spectral Density)
        ###############################################################################
        
        # Construct file name and path
        full_sess_name = str(sub) + "_" + str(sess)
        subj_fname = full_sess_name + "_epo.fif"
        full_path = os.path.join(epoch_load_path, subj_fname)
        print(full_path)

        # Check if file exists
        if not os.path.exists(full_path):
            print('Current Subject ' + str(sub) + '_' + str(sess) + ' does not exist')
            continue

        # Load epoch .fif file
        epoch = mne.read_epochs(epoch_load_path + '/' + full_sess_name + '_epo.fif', preload=False)

        ## PSD Settings
        FS = epoch.info['sfreq']  # Sampling frequency of signals
        PSD_SETTINGS = {
            'method': 'welch',        # Use Welch's method for PSD estimation
            'window': 'hamm',         # Use Hamming window
            'average': 'median',      # Use median for averaging
            'fmin': .1,               # Minimum frequency of interest (Hz)
            'fmax': 50.,              # Maximum frequency of interest (Hz)
            'n_fft': 1024,            # Number of points in FFT
            'n_overlap': int(FS*0.5), # 50% overlap between segments
            'n_per_seg': int(FS),     # Number of points per segment (1 second)
        }
        
        # Compute Power Spectral Density
        epoch_psd = epoch.compute_psd(**PSD_SETTINGS)
        
        # Reshape into [n_spectra, n_freqs]
        epoch_psd_reshaped = np.squeeze(epoch_psd._data[0, :, :])
        
        ###############################################################################
        # Compute Specparam (using FOOOF - Fitting Oscillations & One Over F)
        ###############################################################################
        # Initialize FOOOFGroup to use for fitting
        fg = FOOOFGroup(*FOOOF_SETTINGS, verbose=False)

        # Fit a group of power spectra with the .fit() method
        fg.fit(epoch_psd.freqs, epoch_psd_reshaped, FIT_RANGE)
        
        # Save model fit results
        fg.save(file_name=str(sub) + '_' + str(sess) + '_fooof_results_fixed',
                file_path=fooof_save_path,
                save_results=True, save_settings=True, save_data=True)

I:/CBD_ASD_EEG/02_Epoch_EEG/Concatenated_10_Check\10002_W00_epo.fif
Reading I:\CBD_ASD_EEG\02_Epoch_EEG\Concatenated_10_Check\10002_W00_epo.fif ...
    Found the data of interest:
        t =       0.00 ...   30004.00 ms
        0 CTF compensation matrices available
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
500.0
Loading data for 1 events and 15003 original time points ...
Effective window size : 2.048 (s)
(1, 19, 102)
(19, 102)
I:/CBD_ASD_EEG/02_Epoch_EEG/Concatenated_10_Check\10002_W00_2_epo.fif
Current Subject 10002_W00_2 does not exist
I:/CBD_ASD_EEG/02_Epoch_EEG/Concatenated_10_Check\10002_W00_3_epo.fif
Current Subject 10002_W00_3 does not exist
I:/CBD_ASD_EEG/02_Epoch_EEG/Concatenated_10_Check\10002_W00_4_epo.fif
Current Subject 10002_W00_4 does not exist
I:/CBD_ASD_EEG/02_Epoch_EEG/Concatenated_10_Check\10002_W08_epo.fif
Reading I:\CBD_ASD_EEG\02_Epoch_EEG\Concatenated_10_Check\10002_W08_epo.fif ...
    Found the dat